In [1]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
!pip install seaborn
 
# For visualization
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.options.display.max_rows = None
pd.options.display.max_columns = None

In [3]:
from fosforml.model_manager.snowflakesession import get_session

my_session = get_session()

table_name = 'DATAANALYSISFINAL24'

sf_df = my_session.sql("select * from {}".format(table_name))

df = sf_df.to_pandas()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16848 entries, 0 to 16847
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   DATE           16848 non-null  object 
 1   DRUG_ID        16848 non-null  object 
 2   CUSTOMER_NAME  16848 non-null  object 
 3   VOLUME         16848 non-null  float64
 4   UNIT_PRICE     16848 non-null  int16  
 5   TOTAL          16848 non-null  float64
 6   Discount%      16848 non-null  int8   
 7   CUSTOMER       16848 non-null  object 
 8   CITY           16848 non-null  object 
 9   REGION         16848 non-null  object 
 10  COUNTRY        16848 non-null  object 
 11  NAME           16848 non-null  object 
 12  SEGMENT        16848 non-null  object 
dtypes: float64(2), int16(1), int8(1), object(9)
memory usage: 1.5+ MB


In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load the data
data = df
# Convert DATE column to datetime and then to numerical format
data['DATE'] = pd.to_datetime(data['DATE'])
data['DATE'] = data['DATE'].map(pd.Timestamp.toordinal)

# Encode categorical columns
label_encoders = {}
categorical_columns = ['DRUG_ID', 'CUSTOMER_NAME', 'CUSTOMER', 'CITY', 'REGION', 'COUNTRY', 'NAME', 'SEGMENT']
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Apply Min-Max scaling
scaler = MinMaxScaler()
scaled_columns = ['UNIT_PRICE','Discount%']
data[scaled_columns] = scaler.fit_transform(data[scaled_columns])

# Split the data into training and testing sets
X = data.drop(columns=['VOLUME'])
y = data['VOLUME']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the Random Forest model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

Mean Squared Error: 0.10573594656379817


In [10]:
from sklearn.metrics import mean_absolute_error, r2_score

# Make predictions and evaluate the model
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'Mean Absolute Error: {mae}')
print(f'R-squared: {r2}')

Mean Squared Error: 0.10573594656379817
Mean Absolute Error: 0.015748545994065815
R-squared: 0.9990880437535383
